# Export tick updates using dedicated service - C#

### Overview
This sample demonstrates how to request from a dedicated raw data service **on-demand** tick by tick data `for a given instrument on multiple data sources`. 

### Inputs/outputs
Tick data extraction sample requires instrument's identifier, date time intervals as per inputs and exports tick by tick data file. 

### Services used
This sample uses *gRPC requests* in order to retrieve tick by tick dataset from the hosted service. The queried endpoint in this script are:
* *TickRawService*: to directly retrieve raw dataset from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

# Run the tick updates extract

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.33.*-pre*"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using System.IO;
using System.Globalization;

using MarketFieldUpdate = Systemathics.Apis.Type.Shared.V1.MarketFieldUpdate;
using TickUpdatesService = Systemathics.Apis.Services.Tick.V1.TickUpdatesService;
using TickUpdatesRequest = Systemathics.Apis.Services.Tick.V1.TickUpdatesRequest;
using TickUpdatesResponse = Systemathics.Apis.Services.Tick.V1.TickUpdatesResponse;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = new Grpc.Core.Metadata();
try { headers = TokenHelpers.GetTokenAsMetaData(); } catch {}

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request
To request *raw tick data* service, we need to specify instrument identifier and a period of time. The following code snippets ensure required objects are created and proceed to request instantiation.

#### 3.1 Instrument selection

In [ ]:
// Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
// The ICE ticker
var ticker = "E:BNP";

#### 3.2 Time period delimitation

The following code setups time constraints

In [ ]:
// Create time intervals
// Full order book data avaialble (sample) : from 2021-11-01 to 2021-11-12
var start = new DateTime(2021, 11, 01);
var end = new DateTime(2021, 11, 01);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the tick quotes request time interval (we are using Google date time format)
// UTC time zone
var timeInterval = new TimeInterval()
{ 
 StartTime = new TimeOfDay { Hours = 12, Minutes = 00, Seconds = 00 },
 EndTime = new TimeOfDay { Hours = 20, Minutes = 30, Seconds = 00 } 
};

The following code creates the market data request and service client:

In [ ]:
// Generate constraints based on the previous space selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

// Generate raw tick data request
var request = new TickUpdatesRequest { Constraints = constraints, Adjustment = false };

// We want to stream the ticks from those two Euronext sources (specific to provider "ICE")
request.Identifiers.Add(new IdentifierAndLevel { Provider = "ICE", Ticker = ticker, Exchange = "EQUITY_L2_973", Level = Level.TradesAndBook });
request.Identifiers.Add(new IdentifierAndLevel { Provider = "ICE", Ticker = ticker, Exchange = "787", Level = Level.Trades });

var serviceClient = new TickUpdatesService.TickUpdatesServiceClient(channel);


### Step 4: Retrieve tick by tick data

Tick by tick data is sent through a *stream*, the following code snippet exports tick by tick dataset in a file in the same folder as this sample:

In [ ]:
// Get the tick raw data
var call = serviceClient.TickUpdates(request, headers);

// Export tick raw data to a txt file
const string Header = "TimeStamp,Type,Mapping,Action,Side,Price,Size,Depth,OrderId,Condition,Reset,FieldName,FiedType,FieldValue,MessageNumber";     
var messageNumber = 0;
var ticksCount = 0;
var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
using (var fs = new StreamWriter(File.OpenWrite($"output/{ticker}_tick_updates_multi.csv")))
await foreach (var item in call.ResponseStream.ReadAllAsync())
{
    ++messageNumber;

    if (item.Mapping != null)
    {
        await fs.WriteLineAsync(Header);
        continue;
    }

    if (item.MblBookUpdates != null)
    {
        var itemMblBookUpdates = item.MblBookUpdates;
        foreach (var mblMarketBookUpdate in item.MblBookUpdates.Updates)
        {
            ticksCount++;
            await fs.WriteLineAsync($"{itemMblBookUpdates.TimeStamp.ToDateTime():O},MBL,{itemMblBookUpdates.Mapping},{mblMarketBookUpdate.Action},{mblMarketBookUpdate.Side},{mblMarketBookUpdate.Price},{mblMarketBookUpdate.Size},{mblMarketBookUpdate.Depth},,,{(itemMblBookUpdates.Reset ? "true" : string.Empty)},,,,{messageNumber}");
        }
    }

    if (item.MboBookUpdates != null)
    {
        var itemMboBookUpdates = item.MboBookUpdates;
        foreach (var mboMarketBookUpdate in item.MboBookUpdates.Updates)
        {
            ticksCount++;
            await fs.WriteLineAsync($"{itemMboBookUpdates.TimeStamp.ToDateTime():O},MBO,{itemMboBookUpdates.Mapping},{mboMarketBookUpdate.Action},{mboMarketBookUpdate.Side},{mboMarketBookUpdate.Price},{mboMarketBookUpdate.Size},,{mboMarketBookUpdate.OrderId},{mboMarketBookUpdate.Condition},{(itemMboBookUpdates.Reset ? "true" : string.Empty)},,,,{messageNumber}");
        }
    }

    if (item.FieldsUpdates != null)
    {
        var itemFieldsUpdates = item.FieldsUpdates;
        foreach (var marketFieldUpdate in item.FieldsUpdates.Updates)
        {
            ticksCount++;
            object fieldValue = null;
            var fieldType = string.Empty;
            switch (marketFieldUpdate.ValueCase)
            {
                case MarketFieldUpdate.ValueOneofCase.StringValue:
                    fieldType = "string";
                    fieldValue = marketFieldUpdate.StringValue;
                    break;

                case MarketFieldUpdate.ValueOneofCase.LongValue:
                    fieldType = "long";
                    fieldValue = marketFieldUpdate.LongValue;
                    break;

                case MarketFieldUpdate.ValueOneofCase.DoubleValue:
                    fieldType = "double";
                    fieldValue = marketFieldUpdate.DoubleValue;
                    break;
            }

            await fs.WriteLineAsync($"{itemFieldsUpdates.TimeStamp.ToDateTime():O},FLD,{itemFieldsUpdates.Mapping},{marketFieldUpdate.Action},,,,,,,{(itemFieldsUpdates.Reset ? "true" : string.Empty)},{marketFieldUpdate.Field},{fieldType},{fieldValue},{messageNumber}");
        }
    }
}

The following code snippet is a summary with some metrics about the dateset export: 

In [ ]:
Console.WriteLine($"Start date: {start:yyyy-MM-dd}");
Console.WriteLine($"Start date: {end:yyyy-MM-dd}");
Console.WriteLine($"Total messages retrieved for {ticker}: {messageNumber:N0}");
Console.WriteLine($"Total ticks retrieved for {ticker}: {ticksCount:N0}");